In [1]:
from tabpfn import TabPFNClassifier
from sklearn.ensemble import RandomForestClassifier

import experiments.runners as runners
from data import load_us_perm_visas, load_credit_default

rf_model = RandomForestClassifier()
pfn_model = TabPFNClassifier()

In [2]:
# Instatiate run class that provides functions for training and evaluation
new_run = runners.BasicRun(rf_model, load_credit_default)

Loading data...
Loaded 30000 samples with 23 features and 2 unique classes.
Splitting data for RandomForestClassifier...
Using standard train_test_split (max_length is None)
Setting up pipeline for RandomForestClassifier...
RandomForest model detected.


In [3]:
# Inspect training data
new_run.X_train

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
ID,,,,,,,,,,,,,,,,,,,,,
2997,210000,2,2,1,27,2,2,0,0,0,...,161573,122362,116243,108414,16,6800,3800,5134,4000,4000
19276,210000,2,1,1,33,-2,-2,-2,-2,-2,...,947,500,1283,500,6087,947,500,1283,500,500
4130,170000,2,1,1,52,-1,-1,2,-1,-1,...,416,7509,6387,1516,3000,5,7509,6387,1666,3023
1424,200000,2,3,2,27,1,-2,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
16347,70000,2,1,1,45,1,2,-1,2,-1,...,3060,780,2079,0,0,3450,0,2079,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25781,200000,2,2,1,32,0,0,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
13922,120000,2,2,2,24,0,0,0,0,0,...,115515,113948,122127,121962,4200,4100,4100,10000,4560,0
3795,120000,2,1,2,24,0,0,0,0,0,...,67187,49924,33188,19826,3700,2023,2016,2000,1200,1000


In [4]:
# Inspect pipeline
new_run.pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('datetime',
                                                  DatetimeFeatureEncoder(features=['year',
                                                                                   'month',
                                                                                   'day',
                                                                                   'weekday']),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x325069be0>),
                                                 ('cat_high_c', TargetEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x325077d90>),
                                                 ('num',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x3287f8050>),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x3251949d0>)])),
                ('classifier', RandomForestClassifier())])

In [ ]:
# Ignore FutureWarning, of which TabPFNClassifier has a lot!
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
new_run.fit()

In [ ]:
new_run.predict()
# WARNING: TabPFN takes 50-54 min predicting both class and probability

In [ ]:
new_run.score()

## Work in progress: ROC
Below is code to make the ROC AUC methods work. There is a problem with the array dimensions

In [ ]:
new_run.prob_predictions[:, 0]

In [ ]:
new_run.y_test

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Calculate ROC curve points
fpr, tpr, thresholds = roc_curve(new_run.y_test, new_run.prob_predictions[:, 1], pos_label='certified')

# Calculate AUC score
auc_score = roc_auc_score(new_run.y_test, new_run.prob_predictions[:, 1])
print(f"ROC AUC: {auc_score}")

In [ ]:
# plot ROC curve
import matplotlib.pyplot as plt
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (area = {auc_score:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")